<a href="https://colab.research.google.com/github/Alecia113/NLP-Lab/blob/main/lab2_learnbyself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word2Vec** 请直接粘贴老师代码谢谢

In [1]:
#开始import
'''
import pprint
import re

from lxml import etree  # for parsing Our XML data  用于解析我们的XML数据。

数据存储数据分析
可扩展标记语言（标准通用标记语言的子集）是一种简单的数据存储语言。
XML文件一般指里面写有可扩展标记语言的文件。 XML:可扩展标记语言,标准通用标记语言的子集,是一种用于标记电子文件使其具有结构性的标记语言。它被设计用来传输和存储数据。 XML,是Extensible Markup Language 的缩写。在.NET框架中XML是非常重要的一部分...”

import nltk  # data precessing
nltk.download('punkt')  #很固定
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec  # implementing the word2vec family of algorithms.
#用于实现word2vec 系列算法 ms。

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning) 

'''
import pprint
import re

# For parsing our XML data
from lxml import etree 

# For data processing
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

# For implementing the word2vec family of algorithms
from gensim.models import Word2Vec

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#https://www.cnblogs.com/leomei91/p/7660928.html
#https://blog.csdn.net/weixin_42214743/article/details/107720074
#有些警告是可以忽略的
'''
调用模型还是调整参数

import warnings
warnings.filterwarnings('ignore')

具体函数：
warnings.filterwarnings(action, message='', category=Warning, module='', lineno=0, append=False)
'''

"\n调用模型还是调整参数\n\nimport warnings\nwarnings.filterwarnings('ignore')\n\n具体函数：\nwarnings.filterwarnings(action, message='', category=Warning, module='', lineno=0, append=False)\n"

**然后开始下载数据** use TED script(脚本） from google drive

In [2]:
# 下面代码是： generate a link +  a field (字段) ---> verification code (enter)
#   click link  go to google sign in ; ; (own google account) 
#  copy  verification code + into field + press enter
'''
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth  #GoogleAuthenticator(谷歌身份验证器)
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials  #谷歌  全权证书

#Authenticate  [ɔːˈθentɪkeɪt]  v  鉴定，验证（凹三忒忒咳）
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#看看是不是这个 也不是
'''
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


下载TED Scripts from GD

In [3]:
#点击files tab + see file is download successfully
'''
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('ted_en-20160408.xml')
'''
#在路径中就会过个/content/ted_en-20160408.xml
#好像是这个的问题
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml')  

#尴尬看起来不是这个的问题

**数据处理 data preprocessing**

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')

# Getting contents of <content> tag from the xml file
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# Removing "Sound-effect labels" using regular expression (regex) (i.e. (Audio), (Laughter))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# Tokenising the sentence to process it by using NLTK library
sent_text=sent_tokenize(content_text)

# Removing punctuation and changing all characters to lower case
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# Tokenising each sentence to process individual word
sentences=[]
sentences=[word_tokenize(sentence) for sentence in normalized_text]

# Prints only 10 (tokenised) sentences
print(sentences[:10])



'''
targetXML =open('ted_en-20160408.xml','r',encoding = 'UTF8') #等号后面必须要直接跟open；用utff8来打开ted文件
#open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)

target_text = etree.parse(targetXML)#etree.parse直接接受一个文档，按照文档结构解析（本地文件）
'''
'''
#etree.parse(*args, **kwargs)
#parse(source, parser=None, base_url=None)
'''
'''
#etree.html可以解析html文件：（服务器上返回的html数据）
#https://blog.csdn.net/cbiexi/article/details/104479744?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161568688316780255255960%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161568688316780255255960&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-3-104479744.first_rank_v2_pc_rank_v29&utm_term=etree.parse
#https://blog.csdn.net/myself029/article/details/79954301?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161568688316780255255960%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161568688316780255255960&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-79954301.first_rank_v2_pc_rank_v29&utm_term=etree.parse
#etree 更像是展示出来这个文本文档等等
parse_text = '\n'.join(target_text.xpath('//content/text()')) #获取<内容>标记的内容从xml 文件中【par解析文字】
# 从xml文件中获取<content>标签的内容。
# 【在爬取内容】"\n".join() 来对字符串列表进行处理，不会出现不连贯情况
#https://blog.csdn.net/freeking101/article/details/64461574?utm_medium=distribute.pc_relevant.none-task-blog-OPENSEARCH-1.control&dist_request_id=&depth_1-utm_source=distribute.pc_relevant.none-task-blog-OPENSEARCH-1.control
#https://blog.csdn.net/weixin_39751871/article/details/111537728?ops_request_misc=&request_id=&biz_id=102&utm_term=%27%5Cn%27.join(target_text.xpath(%27/&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-0-111537728.first_rank_v2_pc_rank_v29


content_text = re.sub(r'\([^)]*\)','',parse_text) # 删除音效标签（音频，笑声）#获得文本
'''
'''
\: 将下一个字符————》特殊字符、原义字符。[eg]:\( ==(
^ : 匹配输入字符串的开始位置
*: 匹配前面的子表达式0次或者多次 * == {0,}
'''


'''
#regex 检索词、、正则表达式
#正则表达式强大的文本匹配功能。很多文本匹配处理，如果没有正则表达式，还真的很难做出来。
#https://blog.csdn.net/wyb880501/article/details/79813658

sent_text = sent_tokenize(content_text) #利用NLTK库 对句子进行标记化处理。# 将句子分开。sent_tokenize
#https://blog.csdn.net/wyb880501/article/details/79813658
#regex(蕊·寨·课·嘶)n
#https://blog.csdn.net/Wisimer/article/details/89556404?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161569572316780271589227%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161569572316780271589227&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_click~default-1-89556404.first_rank_v2_pc_rank_v29&utm_term=NLTK
#NLTK 的讲解  

#remove punctuation + change all characters --> lower case  去掉标点符号并将所有字符改为小写。
normalized_text = []     #归一化文本  normalized (孬·某·赖·子·的)
 #从你的语句中获得单词
for string in sent_text: 
  tokens = re.sub(r"[^a-z0-9]+"," ",string.lower()) #[^a-z]负值字符范围;匹配任何不在指定范围内的任意字符;;匹配任何不在 ‘a' 到 ‘z' 范围内的任意字符。 标点符号。
  normalized_text.append(tokens) # 把你处理好的东西丢到列表里。  #这个地方还是句子
#tokenize （偷·可·耐·滋）
# Tokenising (偷·可·耐·滋鹰）符号化 each sentence __process individual word #对每个句子进行符号化处理，以处理单个单词
sentences=[]
sentences=[word_tokenize(sentence) for sentence in normalized_text] #从文本冲取出来然后拆词

#prints only 10 tokenised sentences  
print(sentences[:10])

#这个也不是
'''

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing'], ['consider', 'facit'], ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them'], ['facit', 'was', 'a', 'fantastic', 'company'], ['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world'], ['everybody', 'used', 'them'], ['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along'], ['they', 'continued', 'doing', 'exactly', 'the', 'same']]


'\n#regex 检索词、、正则表达式\n#正则表达式强大的文本匹配功能。很多文本匹配处理，如果没有正则表达式，还真的很难做出来。\n#https://blog.csdn.net/wyb880501/article/details/79813658\n\nsent_text = sent_tokenize(content_text) #利用NLTK库 对句子进行标记化处理。# 将句子分开。sent_tokenize\n#https://blog.csdn.net/wyb880501/article/details/79813658\n#regex(蕊·寨·课·嘶)n\n#https://blog.csdn.net/Wisimer/article/details/89556404?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161569572316780271589227%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161569572316780271589227&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_click~default-1-89556404.first_rank_v2_pc_rank_v29&utm_term=NLTK\n#NLTK 的讲解  \n\n#remove punctuation + change all characters --> lower case  去掉标点符号并将所有字符改为小写。\nnormalized_text = []     #归一化文本  normalized (孬·某·赖·子·的)\n #从你的语句中获得单词\nfor string in sent_text: \n  tokens = re.sub(r"[^a-z0-9]+"," ",string.lower()) #[^a-z]负值字符范围;匹配任何不在指定范围内的任意字符;;匹配任何不在 ‘a\' 到 ‘z\' 范围内的任意字符。 标点符号。\n  normalized_text.append

# **CBOW ——continuous bag of Words** Word2Vec 连续单词袋

有关gensim.models.word2vec的更多详细信息，您可以参考Gensim Word2Vec的API。

In [14]:
#有关gensim.models.word2vec的更多详细信息，您可以参考Gensim Word2Vec的API。
#https://radimrehurek.com/gensim/models/word2vec.html

wv_cbow_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5 , workers=2, sg=0)

'''
Word2Vec(sentences=None, corpus_file=None, size=100, alpha=0.025, window=5, 
min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, 
min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, 
hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, 
sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=(), max_final_vocab=None)
'''
'''
 Initialize and train a word2vec model with the following parameters:
 初始化+训练w2v模型
# sentence: iterable of iterables, i.e. the list of lists of tokens from our data
可迭代的迭代： 符号化来自我们数据的列表们的列表
# size: dimensionality of the word vectors
单词向量的维度
# window: window size
# min_count: ignores all words with total frequency lower than the specified count value
频率比具体化数值低的单词都忽略。 （忽略总频率低语指定计数值的所有单词）

# workers: Use specified number of worker threads （线程；主题）【嘶·若案·子】 to train the model (=faster training with multicore machines)

运行程序就是进程 进程中最小单位是一个线程； 一个进程有至少一个线程
清理垃圾之后再杀毒 是有顺序的，~~单线程 
#https://blog.csdn.net/csdnnews/article/details/82321777?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161569868516780264066483%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161569868516780264066483&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-3-82321777.first_rank_v2_pc_rank_v29&utm_term=线程
还讲了线程安全

【worker是什么含义每天搞懂】使用指定数量的worker 线程来训练模型（=使用multicore机器进行更快的训练）

# sg: training algorithm, 0 for CBOW, 1 for skip-gram
sg：训练算法，CBOW 为0 ； Skip-gram 为1
'''
#这个地方空格了后面的地方没空格

'\n Initialize and train a word2vec model with the following parameters:\n 初始化+训练w2v模型\n# sentence: iterable of iterables, i.e. the list of lists of tokens from our data\n可迭代的迭代： 符号化来自我们数据的列表们的列表\n# size: dimensionality of the word vectors\n单词向量的维度\n# window: window size\n# min_count: ignores all words with total frequency lower than the specified count value\n频率比具体化数值低的单词都忽略。 （忽略总频率低语指定计数值的所有单词）\n\n# workers: Use specified number of worker threads （线程；主题）【嘶·若案·子】 to train the model (=faster training with multicore machines)\n\n运行程序就是进程 进程中最小单位是一个线程； 一个进程有至少一个线程\n清理垃圾之后再杀毒 是有顺序的，~~单线程 \n#https://blog.csdn.net/csdnnews/article/details/82321777?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161569868516780264066483%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161569868516780264066483&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-3-82321777.first_rank_v2_pc_rank_v29&utm_term=线程\n还讲了线程安全\n\n【worker是什么含义每天搞懂】使用指

In [15]:
similar_words = wv_cbow_model.wv.most_similar("man") #topn = 10 default (的A·佛奥·忒) 默认值
pprint.pprint(similar_words)
# The trained word vectors are stored in a KeyedVectors instance（实例） as model.wv
#训练单词向量存在KeyedVectors 实例作为model.wv ([主宾谓]训练后的词向量以model.wv的形式存在keyedvec实例中)
# Get the top 10 similar words to 'man' by calling most_similar() 
#通过calling most_similar() 得到 前十个相似单词。
#通过调用most_similar() 获取与"man"相似的前十个词
# most_similar() computes cosine similarity between a simple mean of the vectors of the given words and the vectors for each word in the model 
#他是变得。
#most_similar() 计算余弦相似度，在一个简单的被给单词的向量含义和每个在模型中向量的单词
#（计算给定的单词的向量和模型中每个单词的向量的简单均值之间的（ a simple mean of） 余弦相似度。
#【不懂】
#【值是变化的可能是获取的文件不够稳定。所有的值都是变化的。】但是频率差距不大。

[('woman', 0.8470235466957092),
 ('guy', 0.7923800945281982),
 ('lady', 0.7808774709701538),
 ('boy', 0.7683364152908325),
 ('girl', 0.7389755845069885),
 ('gentleman', 0.7324548363685608),
 ('soldier', 0.7201880216598511),
 ('kid', 0.7118163704872131),
 ('philosopher', 0.6646140217781067),
 ('photographer', 0.6628036499023438)]


In [16]:
'''
#sg=0；；cbow 
wv_cbow_model = Word2Vec(sentences = sentences, size = 100, window = 5, min_count = 5 , workers = 2, sg = 0)
similar_words = wv_cbow_model.wv.most_similar("man") 

#sg=1；； sg 模型
wv_sg_model = Word2Vec(sentences=sentences, size=100,window=5,min_count=5,workers=2,sg=1)
similar_words=wv_sg_model.wv.most_similar("man") 

ft_sg_model = FastText(sentences,size=100,window=5,min_count=5,workers=2,sg=1)
result=ft_sg_model.wv.most_similar("electrofishing")

ft_cbow_model = FastText(sentences,size=100,window=5,min_count=5,workers=2,sg=0)
result=ft_cbow_model.wv.most_similar("electrofishing")


#就是把（）中的单词变成了K-M+W （）是变化的其他是不变的
'''



'\n#sg=0；；cbow \nwv_cbow_model = Word2Vec(sentences = sentences, size = 100, window = 5, min_count = 5 , workers = 2, sg = 0)\nsimilar_words = wv_cbow_model.wv.most_similar("man") \n\n#sg=1；； sg 模型\nwv_sg_model = Word2Vec(sentences=sentences, size=100,window=5,min_count=5,workers=2,sg=1)\nsimilar_words=wv_sg_model.wv.most_similar("man") \n\nft_sg_model = FastText(sentences,size=100,window=5,min_count=5,workers=2,sg=1)\nresult=ft_sg_model.wv.most_similar("electrofishing")\n\nft_cbow_model = FastText(sentences,size=100,window=5,min_count=5,workers=2,sg=0)\nresult=ft_cbow_model.wv.most_similar("electrofishing")\n\n\n#就是把（）中的单词变成了K-M+W （）是变化的其他是不变的\n'

# Word2Vec-skip Gram

In [5]:
'''
wv_sg_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=2, sg=1)
# KG 和CBOW不同的是sg这个参数多了 从0变1 这样就从CBOW 模型切换到SG模型了。
 #跟空格有关系吗？？没关系
 '''
 # Now we switch to a Skip Gram model by setting parameter sg=1
wv_sg_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=2, sg=1)
#不是上面是这出了问题 不是这个出了问题 与空格无关系

In [7]:
similar_words=wv_sg_model.wv.most_similar("man") #唯一不同把调用的模型名字变了。
pprint.pprint(similar_words)

[('woman', 0.7677208185195923),
 ('guy', 0.7676129937171936),
 ('boy', 0.7084762454032898),
 ('soldier', 0.7077591419219971),
 ('gentleman', 0.6993050575256348),
 ('girl', 0.6862406730651855),
 ('son', 0.6764705181121826),
 ('rabbi', 0.6757755279541016),
 ('michelangelo', 0.6683143973350525),
 ('imam', 0.662891685962677)]


# Word2Vec vs FastText

Word2Vec-Skip Gram 不能发现相似文字去’electrofishing‘ 电子钓鱼；因为电子钓鱼不再词汇表中。

In [ ]:
similar_words=wv_sg_model.wv.most_similar("electrofishing")
pprint.pprint(similar_words)

# FastText-Skip Gram

In [20]:
from gensim.models import FastText  #gensim.models 遗传因子模型
'''
gensim是一款强大的自然语言处理工具，里面包括N多常见模型：

基本的语料处理工具
LSI
LDA
HDP
DTM
DIM
TF-IDF
word2vec、paragraph2vec
.
#gensim详解 https://blog.csdn.net/sinat_26917383/article/details/69803018?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161572013816780262535820%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=161572013816780262535820&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-6-69803018.first_rank_v2_pc_rank_v29&utm_term=gensim.models+用法
【好慢啊……】


model=gensim.models.Word2Vec(sentences,sg=1,size=100,window=5,min_count=2,negative=3,sample=0.001,hs=1,workers=4)
 
#该步骤也可分解为以下三步（但没必要）：
#model=gensim.model.Word2Vec() 建立一个空的模型对象
#model.build_vocab(sentences) 遍历一次语料库建立词典
#model.train(sentences) 第二次遍历语料库建立神经网络模型
 
#sg=1是skip—gram算法，对低频词敏感，默认sg=0为CBOW算法
#size是神经网络层数，值太大则会耗内存并使算法计算变慢，一般值取为100到200之间。
#window是句子中当前词与目标词之间的最大距离，3表示在目标词前看3-b个词，后面看b个词（b在0-3之间随机）
#min_count是对词进行过滤，频率小于min-count的单词则会被忽视，默认值为5。
#negative和sample可根据训练结果进行微调，sample表示更高频率的词被随机下采样到所设置的阈值，默认值为1e-3,
#negative: 如果>0,则会采用negativesamping，用于设置多少个noise words
#hs=1表示层级softmax将会被使用，默认hs=0且negative不为0，则负采样将会被选择使用。
#workers是线程数，此参数只有在安装了Cpython后才有效，否则只能使用单核
 
# min_count，是去除小于min_count的单词
# size，神经网络层数
# sg， 算法选择
# window， 句子中当前词与目标词之间的最大距离
# workers，线程数
#model.save("文本名")	#模型会保存到该 .py文件同级目录下，该模型打开为乱码


和上面那个是一个包

'''
ft_sg_model = FastText(sentences,size=100,window=5,min_count=5,workers=2,sg=1)

In [21]:
result=ft_sg_model.wv.most_similar("electrofishing") #.wv.most_similar这个是固定的前面的模块不同
pprint.pprint(result) #截断的
'''
skip gram 架构（sg=1) 初始化和训练fasttext
fasttext允许我们获取词汇外单词的单词向量
'''

[('electrolux', 0.80483078956604),
 ('electro', 0.7881948947906494),
 ('electrolyte', 0.7861843109130859),
 ('airbus', 0.7736728191375732),
 ('electroshock', 0.7685168981552124),
 ('airbag', 0.7656456232070923),
 ('electrochemical', 0.7533538341522217),
 ('electric', 0.7525812387466431),
 ('electrons', 0.751227080821991),
 ('electrogram', 0.7504618763923645)]


'\nskip gram 架构（sg=1) 初始化和训练fasttext\nfasttext允许我们获取词汇外单词的单词向量\n'

# FastText-CBOW (continuous bag-of-words)

In [22]:
#initialize + train FastText with CBOW architecture(sg=0)
#现在我们使用CBOW的构架初始化和训练FastText （sg=0)  
ft_cbow_model = FastText(sentences,size=100,window=5,min_count=5,workers=2,sg=0)
#[训练依旧很慢]

In [23]:
result=ft_cbow_model.wv.most_similar("electrofishing") #不训练的话；就会：AttributeError: 'FastText' object has no attribute 'wv_most_similar
pprint.pprint(result)
#这里是wv。most-

[('electric', 0.9249635934829712),
 ('electro', 0.9110923409461975),
 ('electrolux', 0.8995359539985657),
 ('electron', 0.8837720155715942),
 ('electronic', 0.8823838233947754),
 ('electrolyte', 0.8793928623199463),
 ('electrical', 0.8793478012084961),
 ('electroshock', 0.8777401447296143),
 ('electromagnet', 0.8753716349601746),
 ('electrode', 0.8738146424293518)]


# **King -Man +Woman = ? **

**Try  both CBOW + SKIP Gram model compute "k-M +W **

得到的词的不一样呢

In [24]:
# sepcify positive/negative word list ~~~ positive/negative parameters
#我们可以使用正/负参数指定正/负词列表
# Top N most similar words can be specified with topn parameter
#可以使用topn参数是定前N个最相似的单词。
result = wv_cbow_model.wv.most_similar(positive = ['woman','king'],negative =['man'],topn=1)
#正的就是positive 负的就是negative
print(result) #指定前1个最相似的

[('president', 0.7811003923416138)]


In [6]:
'''
result = wv_sg_model.wv.most_similar(positive=['woman','king'],negative=['man'],topn=1)
print(result) #wv_sg   #这个地方单词输入很大啊。也不知道怎么了；也不是这句话的问题
'''
result = wv_sg_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('luther', 0.6677122712135315)]


In [26]:
result = ft_cbow_model.wv.most_similar(positive=['woman','king'],negative=['man'],topn=1)
print(result) #ft_cbow 

[('kidding', 0.8879081010818481)]


In [27]:
result = ft_sg_model.wv.most_similar(positive=['woman','king'],negative=['man'],topn=1)
print(result)  #ft_sg  sg

[('pauling', 0.701784610748291)]


没有足够的数据来回答“女王”，下面我们将用更大的数据进行训练。（google 已经使用了google news 数据训练了Word2Vec

# Using Pretrained word embeddings with Gensim ；在Gensim中使用预训练词嵌入

从google预训练的word2Vec二进制文件下载并加载

In [28]:
#https://code.google.com/archive/p/word2vec/  #连接到项目
# Download the pre-trained vectors trained on part of Google News dataset (about 100 billion words)
# Beware, this file is big (3.39GB) - might be long waiting!
# 1000亿个单词  下载在google 新闻数据集的一部分中 经过训练的预训练向量。（3.39GB）
id2 = '0B7XkCwpI5KDYNlNUTTlSS21pQmM'
downloaded = drive.CreateFile({'id':id2})
downloaded.GetContentFile('GoogleNews-vectors-negative300.bin.gz') 

In [29]:
#Uncompress the downloaded file 解压下载的文件。
! gzip -d /content/GoogleNews-vectors-negative300.bin.gz
#zip: /content/GoogleNews-vectors-negative300.bin already exists; do you wish to overwrite (y or n)?
#zip。/content/GoogleNews-vectors-negative300.bin已经存在，你要不要覆盖（y或n）？

In [30]:
from gensim.models import KeyedVectors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#load pretrained vectors with keyedvectors instance(example eg) : might be long waiting!
filename = 'GoogleNews-vectors-negative300.bin'
gn_mv_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
#以姿态将KeyedVectors 加载到预训练的向量中-可能要等很久
#851-52

In [31]:
# now we can try calculate "King-man+woman = ?" again
result = gn_mv_model.most_similar(positive=['woman','king'], negative=['man'], topn=1)
print(result)
#还真是ram不够用中断了
#老师[（'queen'，0.7118192911148071）]
#这个结果就正确了

KeyboardInterrupt: ignored

# load a pretained word embedding model using API ; 

> 使用API加载预训练的词嵌入模型

> another way: use Gensim to load pretrained word

> try GloVe embedding trained on twitter data尝试对Twitter数据进行训练的GloVe嵌入







In [ ]:
import gensim.downloader as api
 
model = api.load("glove-twitter-25") #download model + return as object ready for use(下载模型备用；作为对象返回)
print(model.similarity("cat","dog"))#similarity function can calculate the cosine similarity between 2 give words.
#similarity() 可以计算两个给定单词的余弦相似度
#99.3% 104.1/104.8MB downloaded0.95908207   0.040917932987213135 老师的
#为什么没加载完呢？
#0.95908207
print(model.distance("cat","dog"))
# a another way calculating the similarity between 2 given words ; returns 1-cosine  similarity instead 
'''
0.95908207
0.040917932987213135
'''
#就算是没加载完结果还是一样。而且会用到RAM将近10个G
#第二个是返回1-cosine 相似度
#余弦距离 = 1 - 余弦相似度，其取值范围为[ 0， 2 ]，即相同的两个向量余弦距离为0
'''
欧氏距离体现数值上的绝对差异，而余弦距离体现方向上的相对差异。
余弦距离不是严格定义上的距离！
https://blog.csdn.net/m0_38024592/article/details/100075613?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161575999916780265455768%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161575999916780265455768&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~baidu_landing_v2~default-1-100075613.first_rank_v2_pc_rank_v29&utm_term=1-cosine+


'''


# 【tips】 paly with colab form fields 玩colab表单字段。

'>'表示换行 ’enter‘表示空格

> form supports multiple types of fields（字段）。 【 input fields； dropdown menus】 表单支持多种类型的字段； 【输入字段；下拉菜单】

上个作业的E1 就是 你可以用double-clicking it 

changing valure in each input field (right) + check changes in the code(left) _vice versa
代码对应文本






In [ ]:
#@title example form fields
#@markdown please put description 请输入描述
string = "examples" #@param {type:"string"}
#表单汇总添加字段（右键最下面表单）
# 只要加上花A功能块  后面的东西就可以显示出来。
slider_value = 100 #@param{type:"slider", min:100, max:200}
#slider_value 是个滚轴区间范围的滚轴；移动滚轴数值就在变化
number =  1231231#@param{type:"number"}
 #数字而且（在警号花param之后不能在有注释了）
data = '2020-01-05'#@param{type:"date"}
pick_me = "monday" #@param['monday','tuesday','wednesday','thursday']
select_or_input = "\u6709\u70B9\u50CF\u8F93\u5165\u5355\u8BCD\uFF0C\u7136\u540E\u5F39\u51FA\u4EC0\u4E48\u6765" #@param["apples","bananas","oranges"]{allow-input:true}
#选择和输入

'''
param (泼·若啊·母)
Embedded Image 英[ˌpærəˈm] Embedded Image 美[ˌpærəˈm] 

abbr. 参数（Parametric）
date 直接弹出日期选择表单了
#['1','2','3'……]下拉菜单 【""，""，""】也是下拉菜单
'''

#print the output
print("string is",string)
print('slider_value',slider_value)
# 打印的话就是将名字直接放进print就行不需要再加''

In [ ]:
#string is examples
#slider_value 111

**测试TEST**

In [ ]:
print(filename)

In [ ]:
filename

In [ ]:
result = wv_cbow_model.wv.most_similar(positive = ['woman','king'],negative =['man'],topn=10)
result
#和老师的lab结果还不太一样

In [ ]:
print(ft_sg_model) #FastText(vocab=21613, size=100, alpha=0.025)

In [ ]:
ft_sg_model  #<gensim.models.fasttext.FastText at 0x7f23bdeacf50>

In [ ]:
print(wv_cbow_model) #Word2Vec(vocab=21613, size=100, alpha=0.025)

In [ ]:
wv_cbow_model #<gensim.models.word2vec.Word2Vec at 0x7f0a2c87e690>


In [ ]:
sentences[:6]# 5句话，然后单词化了

In [ ]:
normalized_text[:10]

In [ ]:
content_text[:1000] #个数；是字母的个数（获取文本）

In [ ]:
sent_text[:10] #十句话。

老师的


In [1]:
import pprint
import re

# For parsing our XML data
from lxml import etree 

# For data processing
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

# For implementing the word2vec family of algorithms
from gensim.models import Word2Vec

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml')  

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')

# Getting contents of <content> tag from the xml file
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# Removing "Sound-effect labels" using regular expression (regex) (i.e. (Audio), (Laughter))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# Tokenising the sentence to process it by using NLTK library
sent_text=sent_tokenize(content_text)

# Removing punctuation and changing all characters to lower case
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# Tokenising each sentence to process individual word
sentences=[]
sentences=[word_tokenize(sentence) for sentence in normalized_text]

# Prints only 10 (tokenised) sentences
print(sentences[:10])




[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing'], ['consider', 'facit'], ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them'], ['facit', 'was', 'a', 'fantastic', 'company'], ['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world'], ['everybody', 'used', 'them'], ['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along'], ['they', 'continued', 'doing', 'exactly', 'the', 'same']]


In [ ]:
# Initialize and train a word2vec model with the following parameters:
# sentence: iterable of iterables, i.e. the list of lists of tokens from our data
# size: dimensionality of the word vectors
# window: window size
# min_count: ignores all words with total frequency lower than the specified count value
# workers: Use specified number of worker threads to train the model (=faster training with multicore machines)
# sg: training algorithm, 0 for CBOW, 1 for skip-gram
wv_cbow_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=2, sg=0)
#可能是这个地方出问题了。回去看看


In [ ]:
# The trained word vectors are stored in a KeyedVectors instance as model.wv
# Get the top 10 similar words to 'man' by calling most_similar() 
# most_similar() computes cosine similarity between a simple mean of the vectors of the given words and the vectors for each word in the model 

similar_words=wv_cbow_model.wv.most_similar("man") # topn=10 by default
pprint.pprint(similar_words)

In [5]:
# Now we switch to a Skip Gram model by setting parameter sg=1
wv_sg_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=2, sg=1)
#不是上面是这出了问题 不是这个出了问题 与空格无关系

In [ ]:
similar_words=wv_sg_model.wv.most_similar("man")
pprint.pprint(similar_words)

In [ ]:
from gensim.models import FastText

In [ ]:
# Now we initialize and train FastText with Skip Gram architecture (sg=1)
ft_sg_model = FastText(sentences, size=100, window=5, min_count=5, workers=2, sg=1)

In [ ]:
# As we can see, FastText allows us to obtain word vectors for out-of-vocabulary words
result=ft_sg_model.wv.most_similar("electrofishing")
pprint.pprint(result)

In [ ]:
# Now we initialize and train FastText with CBOW architecture (sg=0)
ft_cbow_model = FastText(sentences, size=100, window=5, min_count=5, workers=2, sg=0)

In [ ]:
# Again, FastText allows us to obtain word vectors for out-of-vocabulary words
result=ft_cbow_model.wv.most_similar("electrofishing")
pprint.pprint(result)

In [ ]:
# We can specify the positive/negative word list with the positive/negative parameters
# Top N most similar words can be specified with the topn parameter
result = wv_cbow_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)


In [9]:
result = wv_sg_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)
#queen

[('luther', 0.6583076119422913)]


In [ ]:
result = ft_cbow_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)


In [ ]:
result = ft_sg_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)
#hawking